In [21]:
import numpy as np
import pandas as pd
import pylab as pl

from sklearn.linear_model import Ridge

prediction_files = ['prediction_rf.csv', 'prediction_weights.csv', 'prediction_ridge.csv', 'prediction_ens_rf.csv']

df_prediction = [pd.read_csv(file) for file in prediction_files]

def get_stacked(df_prediction):
    df_stack = df_prediction[0].rename(columns={'Sales': 'Sales_0'})
    for i in range(1, len(df_prediction)):
        df_stack = pd.merge(df_stack, df_prediction[i].rename(columns={'Sales': 'Sales_' + str(i)}), on='Id')
    return df_stack

get_stacked(df_prediction)

,Id,Sales_0,Sales_1,Sales_2,Sales_3
0,1,4334.378070,4787.007214,4176.768576,4468.440867
1,2,7686.154676,8080.993636,7758.013861,7870.083437
2,3,9525.090339,10346.925177,9652.443704,9405.143000
3,4,7790.480961,7717.238893,8598.911658,8009.012645
4,5,7536.729341,7604.183504,8057.198036,7777.313266
5,6,5893.576401,6030.074381,5962.290136,5921.263673
6,7,7832.651976,8351.561036,7797.399921,7643.246639
7,8,8542.237411,8595.822056,8649.256996,8445.055623
8,9,5670.529577,5877.829757,5598.523072,5690.947972
9,10,5831.200083,6102.277618,5956.811516,5733.850330


In [29]:
df_prediction = [pd.read_csv(file) for file in prediction_files]
    
coeffs = np.array([1.5, 0.5, 0.5, 1.5])

print('Means ', [df.Sales.mean() for i, df in enumerate(df_prediction)])
print('coeffs', coeffs)

assert coeffs.sum() == len(coeffs)

for i in range(len(df_prediction)):
    df_prediction[i].Sales = coeffs[i]*df_prediction[i].Sales
    
df_stack = get_stacked(df_prediction)
df_stack['Sales'] = df_stack[['Sales_' + str(i) for i in range(len(df_prediction))]].mean(axis=1)

df_stack[['Id','Sales']].to_csv('prediction.csv', index=False)
df_stack.Sales

Means  [5881.406364289439, 5972.760133056349, 5867.001751621936, 5920.685859900287]
coeffs [ 1.5  0.5  0.5  1.5]


0         4421.529075
1         7813.465229
2         9598.758612
3         7964.328921
4         7700.438670
5         5929.610592
6         7822.082100
7         8525.869769
8         5695.098185
9         5844.280047
10        7595.927754
11        8313.069500
12        7396.355854
13        9733.318054
14        6049.099523
15        5056.677485
16        5946.970932
17       10247.168647
18       11126.531013
19       10395.283613
20        8129.142080
21        4832.205364
22        6117.253710
23        4598.631213
24        7793.143196
25       11766.868084
26        9955.147193
27        6049.292033
28        5593.469035
29        4830.041387
             ...     
41058     2854.702355
41059     6209.608334
41060     6616.205112
41061    10985.493685
41062     6015.930571
41063     5759.828404
41064     3904.061142
41065     6586.639120
41066     3104.227758
41067    10597.106622
41068     7501.612486
41069    12967.870933
41070     5804.876242
41071     5303.571003
41072     

In [ ]:
del df_stack, df_prediction